In [69]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as df
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [71]:
# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# end_date = '2024-11-25'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list, 
#                  start=start_date,
#                  end=end_date).stack()

# df.index.names = ['date', 'ticker']

# df.columns = df.columns.str.lower()

# df

[*********************100%***********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-11-28 A        40.981754   43.599998   44.689999   43.580002   44.520000   
           AAPL     25.831564   27.892500   28.117500   27.847500   27.857500   
           ABBV     42.172958   59.459999   59.939999   59.009998   59.130001   
           ABT      33.519829   38.740002   39.220001   38.700001   39.220001   
           ACGL     25.870779   27.206667   27.406668   27.116667   27.343332   
...                       ...         ...         ...         ...         ...   
2024-11-22 XYL     126.870003  126.870003  127.730003  125.760002  126.139999   
           YUM     135.270004  135.270004  135.869995  134.179993  134.529999   
           ZBH     110.220001  110.220001  110.370003  108.669998  108.669998   
           ZBRA    397.190002  397.190002  397.839996  392.540009  394.279999   
           ZTS     176.960007  176.960007  178.070007  176.279999  176.350006   

Price                   volume  
date       ticker               
2016-11-28 A         2241800.0  
           AAPL    108776000.0  
           ABBV      6274600.0  
           ABT       5727800.0  
           ACGL      1845000.0  
...                        ...  
2024-11-22 XYL       1365600.0  
           YUM        992500.0  
           ZBH       2015000.0  
           ZBRA       232000.0  
           ZTS       1854600.0  

[989262 rows x 6 columns]

## Calculate features and technical indicators for each stock.
##### Garman-Klass Volatlity
##### RSI
##### Bollinger Bands
##### ATR
##### MACD 
##### Dollar Volume

In [72]:
# df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# def compute_atr(stock_data):
#    atr = pandas_ta.atr(high=stock_data['high'],
#                        low=stock_data['low'],
#                        close=stock_data['close'],
#                       length=14)
#    return atr.sub(atr.mean()).div(atr.std())

# df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# def compute_macd(close):
#    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
#    return macd.sub(macd.mean()).div(macd.std())

# df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

# df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-11-28 A        40.981754   43.599998   44.689999   43.580002   44.520000   
           AAPL     25.831564   27.892500   28.117500   27.847500   27.857500   
           ABBV     42.172958   59.459999   59.939999   59.009998   59.130001   
           ABT      33.519829   38.740002   39.220001   38.700001   39.220001   
           ACGL     25.870779   27.206667   27.406668   27.116667   27.343332   
...                       ...         ...         ...         ...         ...   
2024-11-22 XYL     126.870003  126.870003  127.730003  125.760002  126.139999   
           YUM     135.270004  135.270004  135.869995  134.179993  134.529999   
           ZBH     110.220001  110.220001  110.370003  108.669998  108.669998   
           ZBRA    397.190002  397.190002  397.839996  392.540009  394.279999   
           ZTS     176.960007  176.960007  178.070007  176.279999  176.350006   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-11-28 A         2241800.0         -0.002333        NaN       NaN   
           AAPL    108776000.0         -0.002156        NaN       NaN   
           ABBV      6274600.0         -0.043999        NaN       NaN   
           ABT       5727800.0         -0.009439        NaN       NaN   
           ACGL      1845000.0         -0.001127        NaN       NaN   
...                        ...               ...        ...       ...   
2024-11-22 XYL       1365600.0          0.000108  48.788606  4.781060   
           YUM        992500.0          0.000067  52.082171  4.883569   
           ZBH       2015000.0          0.000043  54.438181  4.658121   
           ZBRA       232000.0          0.000069  60.907590  5.914542   
           ZTS       1854600.0          0.000046  44.642241  5.147016   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-11-28 A            NaN       NaN       NaN       NaN      91.872897  
           AAPL         NaN       NaN       NaN       NaN    2809.854200  
           ABBV         NaN       NaN       NaN       NaN     264.618445  
           ABT          NaN       NaN       NaN       NaN     191.994875  
           ACGL         NaN       NaN       NaN       NaN      47.731587  
...                     ...       ...       ...       ...            ...  
2024-11-22 XYL     4.829339  4.877619  0.626261 -1.179897     173.253676  
           YUM     4.907857  4.932145  0.712395 -0.225595     134.255479  
           ZBH     4.699571  4.741022 -0.590623  0.465294     222.093302  
           ZBRA    5.964976  6.015410  0.087145  0.531073      92.148081  
           ZTS     5.180613  5.214210  0.660563 -1.215823     328.190028  

[989262 rows x 14 columns]

## Aggregate monthly levels and indicators to filter the top 150 most liquid stocks

In [83]:
#last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

#data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('ME').mean().stack('ticker').to_frame('dollar_volume'),
#           df.unstack()[last_cols].resample('ME').last().stack('ticker')],
#          axis=1).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2017-01-31 A          102.349486   46.162548         -0.000339  61.726303   
           AAPL      3120.691433   28.095905         -0.002159  71.538471   
           ABBV       344.889472   43.780575         -0.038072  49.986848   
           ABT        371.924556   36.374996         -0.004523  72.328483   
           ACGL        29.688113   28.003962         -0.001044  61.679154   
...                          ...         ...               ...        ...   
2024-11-30 XYL        203.596445  126.870003          0.000108  48.788606   
           YUM        270.208410  135.270004          0.000067  52.082171   
           ZBH        180.060551  110.220001          0.000043  54.438181   
           ZBRA       149.927101  397.190002          0.000069  60.907590   
           ZTS        528.385041  176.960007          0.000046  44.642241   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2017-01-31 A       3.803595  3.833158  3.862721 -1.381981  0.136707  
           AAPL    3.327728  3.356231  3.384733 -1.446601 -0.131993  
           ABBV    3.779777  3.810983  3.842188 -1.438255 -0.325409  
           ABT     3.557928  3.591215  3.624503 -1.412058  0.188883  
           ACGL    3.331300  3.355690  3.380080 -1.172477 -0.008442  
...                     ...       ...       ...       ...       ...  
2024-11-30 XYL     4.781060  4.829339  4.877619  0.626261 -1.179897  
           YUM     4.883569  4.907857  4.932145  0.712395 -0.225595  
           ZBH     4.658121  4.699571  4.741022 -0.590623  0.465294  
           ZBRA    5.914542  5.964976  6.015410  0.087145  0.531073  
           ZTS     5.147016  5.180613  5.214210  0.660563 -1.215823  

[46730 rows x 9 columns]

## Calculate 5-year rolling average of dollar volume for each stocks before filtering

In [91]:
#data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

#data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

#data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2021-12-31 AAPL    174.708160         -0.000081  62.210540  5.095859   
           ABBV    120.844727         -0.005397  75.138292  4.673201   
           ABT     133.047501         -0.001274  66.194911  4.815417   
           ACN     396.205444         -0.000575  67.780181  5.826416   
           ADBE    567.059998          0.000030  40.208865  6.267679   
...                       ...               ...        ...       ...   
2024-11-30 VZ       43.150002          0.000209  57.378871  3.713802   
           WFC      75.959999          0.000075  74.391808  4.137442   
           WMT      90.440002          0.000192  74.711070  4.394861   
           XOM     121.790001          0.000082  57.448623  4.751835   
           ZTS     176.960007          0.000046  44.642241  5.147016   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2021-12-31 AAPL    5.151003  5.206146  0.920098  1.740982  
           ABBV    4.753917  4.834632 -0.425083  1.937525  
           ABT     4.865827  4.916237  0.788317  2.287493  
           ACN     5.922422  6.018428  1.512010  2.853608  
           ADBE    6.392441  6.517203  1.956321 -2.492518  
...                     ...       ...       ...       ...  
2024-11-30 VZ      3.747945  3.782089 -0.331598 -0.043189  
           WFC     4.260497  4.383553  1.816726  3.702242  
           WMT     4.447200  4.499539  2.702822  2.779546  
           XOM     4.786490  4.821145  0.427473  0.439478  
           ZTS     5.180613  5.214210  0.660563 -1.215823  

[5364 rows x 8 columns]